In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('clean_data.csv')
data

,Timestamp (UTC),Accel-X,Accel-Y,Accel-Z,Gyro-X,Gyro-Y,Gyro-Z,Mag-X,Mag-Y,Mag-Z
0,2021-07-18 19:38:25.216649,1.026352,0.220828,9.833689,0.582308,0.322654,-0.769382,-2.715035,-12.157078,45.595769
1,2021-07-18 19:38:25.832986,0.970908,0.332507,9.819850,0.592509,0.330452,-0.769902,-2.074577,-11.658333,45.665719
2,2021-07-18 19:38:26.450044,0.903096,0.471929,9.803224,0.589182,0.324757,-0.766122,-1.486484,-11.040606,45.693542
3,2021-07-18 19:38:27.067046,0.849210,0.607099,9.805447,0.587473,0.323649,-0.764833,-0.864133,-10.516000,45.825130
4,2021-07-18 19:38:27.683402,0.623980,0.684462,9.749561,0.576663,0.312049,-0.816623,-0.164515,-10.441316,45.930627
...,...,...,...,...,...,...,...,...,...,...
3065,2021-07-18 20:09:58.359088,0.247233,2.146055,9.135187,0.567664,0.256865,-1.414120,21.768907,10.663329,45.160894
3066,2021-07-18 20:09:58.975162,0.365237,2.047128,9.136237,0.571476,0.281306,-1.323460,22.492522,11.696796,45.049961
3067,2021-07-18 20:09:59.591295,0.193979,1.999785,9.333761,0.553771,0.297137,-1.141288,23.182716,12.848491,44.850635
3068,2021-07-18 20:10:00.208589,0.059440,2.091060,9.186115,0.584193,0.298420,-1.045016,23.697170,13.603377,44.708225


In [3]:
avg_acc_z = data['Accel-Z'].mean()
median_acc_z = data['Accel-Z'].median() 
print("avg vertical acceleration", avg_acc_z)
print("median vertical acceleration", median_acc_z)

avg vertical acceleration 9.743679279894794
median vertical acceleration 9.720214451623715


In [5]:
acc_x = data['Accel-X'].to_numpy()
acc_y = data['Accel-Y'].to_numpy()
acc_z = data['Accel-Z'].to_numpy()

accs = np.vstack((acc_x, acc_y, acc_z))

avg_acc_mag = np.average(np.sqrt(acc_x ** 2 + acc_y ** 2 + acc_z ** 2))
avg_acc_mag = np.average(np.linalg.norm(accs, axis=0)) # ^ does the same thing iin more concise way
print(avg_acc_mag)
median_acc_mag = np.median(np.linalg.norm(accs, axis=0))
print(median_acc_mag)

9.830983591451451
9.839160462711895


In [6]:
# average tilt = theta
# cos(theta) = g / (avg z acceleration)
# theta = arccos(g / (avg z acceleration))
g = 9.80665 # m/s^2
avg_tilt = np.arccos(avg_acc_z / g)
print(avg_tilt * 180 / np.pi)
median_tilt = np.arccos(median_acc_z / g)
print(median_tilt * 180 / np.pi)

6.496497689293469
7.6127749850871655


In [10]:
# v_x[i] = v_x[i-1] + a_x[i]*t
data['Timestamp (UTC)'] = data['Timestamp (UTC)'].apply(pd.to_datetime)
dt = (data['Timestamp (UTC)'][1] - data['Timestamp (UTC)'][0]).total_seconds()
# ^ number of seconds between first time AUV recorded acceleration and the second time

length = data['Accel-X'].shape[0] # length of data
v_x = np.zeros(length) # array of velocities in x-axis
v_y = np.zeros(length) # array of velocities in y-axis
dt = np.zeros(length) # array of velocities in y-axis
print(length + 1)
for i in range(1, length):
    # print(data['Timestamp (UTC)'][i] - data['Timestamp (UTC)'][i-1])
    dt[i] = (data['Timestamp (UTC)'][i] - data['Timestamp (UTC)'][i-1]).total_seconds()
    v_x[i] = v_x[i-1] + data['Accel-X'][i] * dt[i]
    v_y[i] = v_y[i-1] + data['Accel-Y'][i] * dt[i]
print(v_x)
data['Velocity-X'] = v_x
data['Velocity-Y'] = v_y
dx = v_x * dt
dy = v_y * dt
# print(v_x)
# print(v_y)
# print(dt)
# np.savetxt('v_x.txt', v_x)
print(sum(dx))
print(sum(dy))
data

3071
[   0.            0.5984067     1.1556691  ... -215.16748019 -215.13078824
 -215.06885564]
-52178.935964013115
29202.58900819356


,Timestamp (UTC),Accel-X,Accel-Y,Accel-Z,Gyro-X,Gyro-Y,Gyro-Z,Mag-X,Mag-Y,Mag-Z,velocity,Velocity-X,Velocity-Y
0,2021-07-18 19:38:25.216649,1.026352,0.220828,9.833689,0.582308,0.322654,-0.769382,-2.715035,-12.157078,45.595769,0.000000,0.000000,0.000000
1,2021-07-18 19:38:25.832986,0.970908,0.332507,9.819850,0.592509,0.330452,-0.769902,-2.074577,-11.658333,45.665719,0.598407,0.598407,0.204936
2,2021-07-18 19:38:26.450044,0.903096,0.471929,9.803224,0.589182,0.324757,-0.766122,-1.486484,-11.040606,45.693542,1.155669,1.155669,0.496144
3,2021-07-18 19:38:27.067046,0.849210,0.607099,9.805447,0.587473,0.323649,-0.764833,-0.864133,-10.516000,45.825130,1.679634,1.679634,0.870726
4,2021-07-18 19:38:27.683402,0.623980,0.684462,9.749561,0.576663,0.312049,-0.816623,-0.164515,-10.441316,45.930627,2.064228,2.064228,1.292598
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065,2021-07-18 20:09:58.359088,0.247233,2.146055,9.135187,0.567664,0.256865,-1.414120,21.768907,10.663329,45.160894,-215.512010,-215.512010,-358.226834
3066,2021-07-18 20:09:58.975162,0.365237,2.047128,9.136237,0.571476,0.281306,-1.323460,22.492522,11.696796,45.049961,-215.286997,-215.286997,-356.965651
3067,2021-07-18 20:09:59.591295,0.193979,1.999785,9.333761,0.553771,0.297137,-1.141288,23.182716,12.848491,44.850635,-215.167480,-215.167480,-355.733518
3068,2021-07-18 20:10:00.208589,0.059440,2.091060,9.186115,0.584193,0.298420,-1.045016,23.697170,13.603377,44.708225,-215.130788,-215.130788,-354.442719
